In [1]:
import numpy as np
import random 
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
class Bandit:
    
    def __init__(self, k):
        """
        k: number of bandits 
        """
        self.k = k
        self.mean_sd_list = [] # Storing mean and sf of each bandit
        
        max_mean = 0
        self.max_i = 0
        
        for i in range(k):
            mean = np.random.randint(5, 100)
            sigma = random.uniform(0, 1)
            self.mean_sd_list.append((mean, sigma))
            
            if mean > max_mean:
                max_mean = mean
                self.max_i = i
        
    def generate_reward(self, i):
        mu, sigma = self.mean_sd_list[i]
        return np.random.normal(mu, sigma)
    
    def generate_optimum_reward(self):
        return self.generate_reward(self.max_i)

In [3]:
class Environment(object):
    
    def __init__(self, bandit):
        """
        bandit(object of class Bandit) to solve
        """
        
        self.bandit = bandit
        
        self.counts = [0] * self.bandit.k
        self.mean_estimate = [0] * self.bandit.k
        self.actions = []
        
    def reset(self):
        self.counts = [0] * self.bandit.k
        self.mean_estimate = [0] * self.bandit.k
        self.actions = []
        
        init_bandit = np.random.randint(0, self.bandit.k)
#         print("Init_bandit = ", init_bandit)
        return init_bandit
    
    def step(self, a):
        '''
        updates current state and returns next state
        a : chosen bandit index
        '''
#         print("a = ", a)
        reward = self.bandit.generate_reward(a)
        self.mean_estimate[a] = (self.mean_estimate[a] * self.counts[a] + reward)/ (self.counts[a] + 1)
        self.counts[a] += 1
        
        next_bandit = np.argmax(self.mean_estimate)
        
        return next_bandit, reward
        

In [38]:
def main(k, bandit = None, num_episodes = 10, num_trials = 10000, compute_convergence = False, resetQ = False, debug = False, lr = 0.8, y = 0.95):
    """
    k = number of bandits
    """
    
    # defining object of class Bandit 
    if not bandit:
        bandit = Bandit(k) 
        print("Guassian distribution of bandits = \n", bandit.mean_sd_list)
    
    env = Environment(bandit)
    
    Q = np.zeros([k, k])
    reward_list = []
    state_transition_count = np.zeros(k)
    convergence_episode = None
    best_bandit = None
    
    for i in range(num_episodes):
        
        if compute_convergence:
            if resetQ:
                Q = np.zeros([k, k])
            state_transition_count = np.zeros(k)
            best_bandit_so_far = None
            best_bandit_streak = 0
            
        s = env.reset()
            
        total_reward = 0
        j = 0
        if debug:    print("--- Episode {} ---".format(i))
        while j < num_trials:
            j += 1
            
            if np.random.random() < 0.4:
                a = np.random.randint(0, k)
            else:
                a = np.argmax(Q[s, :])
                
            # Get new state and reward from environment
            s1, reward = env.step(a)
            
#             print("\ns = {}, a = {}, s1 = {}, reward = {}".format(s, a, s1, reward))
#             print("Mean estimate = ", env.mean_estimate)
            
            # Update Q-table with new knowledge
            Q[s,a] = Q[s,a] + lr * (reward + y * np.max(Q[s1,:]) - Q[s,a])
            state_transition_count[s] += 1
            
            total_reward += reward
            s = s1
             
            '''
            Trying different convergece conditions
            Method 1 - 
            In each episode, if the method gives the same best bandit for {20} trials, 
            then the algorithm has converged. If the streak is broken, restart the count for the new best bandit. 
            {20} is a hyperparameter. 
            '''
            
#             if compute_convergence:
#                 best_bandits = np.argmax(Q, axis = 1)
                
#                 single_best_bandit = np.all(best_bandits == best_bandits[0])
#                 if single_best_bandit:
#                     if best_bandits[0] == best_bandit_so_far:
#                         best_bandit_streak += 1
#                     else:
#                         best_bandit_so_far = best_bandits[0]
#                         best_bandit_streak = 1
                        
#                     if best_bandit_streak > 20:
#                         print("\n\nQ = {}, \nbest_bandit = {}, \nstate_tranisition_count {}".format(Q, best_bandits, state_transition_count))
#                         print("Convergence at t = ", j)
#                         convergence_episode = i
#                         break

        '''
        Method 2 - 
        After each episode, if all the bandits has the hight Q value for a single bandit, 
        i.e the best decision for any bandit is to move to a single bandit then the algorithm has converged. 
        This method seems more intutive. 
        '''

        if compute_convergence:
            best_bandits = np.argmax(Q, axis = 1)

            single_best_bandit = np.all(best_bandits == best_bandits[0])
            if single_best_bandit:
                convergence_episode = i
                best_bandit = best_bandits[0]
                if debug:
                    print("\n\nQ = {}, \nbest_bandit = {}, \nstate_tranisition_count {}".format(Q, best_bandits, state_transition_count))
                break

        reward_list.append(total_reward) 

#     print("\nQ value Table : \n", Q)
    if debug:
        print("\nScore over time: ", sum(reward_list)/num_episodes)
    return convergence_episode, best_bandit
    

In [50]:
convergence_episode, best_bandit = main(k = 3, num_episodes = 100, num_trials = 10000, compute_convergence = True, resetQ = False, debug = False)
print("Best bandit = {}, Converged at episode {}".format(best_bandit, convergence_episode))

Guassian distribution of bandits = 
 [(8, 0.19687189128846827), (31, 0.7999575530033356), (93, 0.2299658416800454)]
Best bandit = 2, Converged at episode 6


### Reset Q = False

In [40]:
# number of bandits 
k = 3
bandit = Bandit(k) 
print("Guassian distribution of bandits = \n", bandit.mean_sd_list)

Guassian distribution of bandits = 
 [(6, 0.047925764668003024), (93, 0.712428442313467), (96, 0.37456786473436177)]


In [41]:
num_episodes = 100
num_trials = 1000
resetQ = False
avg_over_steps = 10

In [42]:
convergence_episodes_list = []
for _ in range(avg_over_steps):
    convergence_episode, best_bandit = main(k = k, bandit = bandit, num_episodes = num_episodes, num_trials = num_trials, compute_convergence = True, resetQ = resetQ)
    convergence_episodes_list.append(convergence_episode)
    

In [45]:
avg_episodes = np.mean(convergence_episodes_list)
print("Average episodes ({} steps) for {} bandits to converge = {}".format(num_trials, k, avg_episodes))

Average episodes (1000 steps) for 3 bandits to converge = 27.4


### Reset Q = True

In [52]:
num_episodes = 10
num_trials = 30000 # setting num_trials to 27.4 * 1000 = 27400 ~ 30000
resetQ = True
avg_over_steps = 10

In [53]:
convergence_episodes_list = []
for _ in range(avg_over_steps):
    convergence_episode, best_bandit = main(k = k, bandit = bandit, num_episodes = num_episodes, num_trials = num_trials, compute_convergence = True, resetQ = resetQ)
    convergence_episodes_list.append(convergence_episode)
    

In [60]:
if None not in convergence_episodes_list: 
    avg_episodes = np.mean(convergence_episodes_list)
else:
    num_none = 0
    for e in convergence_episodes_list:
        if not e:
            num_none += 0
    if num_none == len(convergence_episodes_list):
        avg_episodes = None
    else:
        avg_episodes = np.mean([x for x in convergence_episodes_list if x is not None])
        
print("Average episodes ({} steps) for {} bandits to converge = {}".format(num_trials, k, avg_episodes))

Average episodes (30000 steps) for 3 bandits to converge = nan


* On resetting the value of Q after each episode, the algorithm does not converge, even on increasing the number of trials and episodes. In the case of bandits, resetting Q implies that the knowledge of transition is lost when the environment is reset (mean estimate of the bandits). 

* One reason could be since the Q value is resetting that more number of trials might be needed in an episode to converge. Increasing num_trials from 1000 to 30000 which is the average steps needed for the algorithm to converge when Q is not resetted. Even after that, the algorithm is not converging. 

* It is interesting to observe that for the iterative Q learning to converge, it is necessary to reset our environment - in this case, losing the knowledge of the bandits by setting the mean estimate of the bandits back to 1. 